In [1]:
import pandas as pd

# Price Data:

### Clean Data:

In [2]:
# Load Price Data
PricesData = pd.read_csv('Data/cardPrices.csv',index_col = 'uuid')

In [3]:
# Filter Mtgo Cards 
filter = PricesData['gameAvailability'] == 'paper'
Prices = PricesData[filter].copy()

In [4]:
# Standarized Prices 
mask = (Prices['currency'] == 'EUR')
Prices_masked = Prices[mask]

Prices['price USD'] = Prices['price']
Prices.loc[mask,'price USD']= Prices_masked['price'] *1.06

In [5]:
# Drop Extraneous Data
Prices.drop(['price','gameAvailability','currency','date'],axis=1,inplace = True)

### Pivot Data

In [6]:
# Pivot Price Data 
foil_retailPrices = Prices.groupby(['providerListing']).get_group('retail').groupby(['cardFinish']).get_group('foil')['price USD'].groupby('uuid').mean()
foil_buylistPrices = Prices.groupby(['providerListing']).get_group('buylist').groupby(['cardFinish']).get_group('foil')['price USD'].groupby('uuid').mean()
normal_retailPrices = Prices.groupby(['providerListing']).get_group('retail').groupby(['cardFinish']).get_group('normal')['price USD'].groupby('uuid').mean()
normal_buylistPrices = Prices.groupby(['providerListing']).get_group('buylist').groupby(['cardFinish']).get_group('normal')['price USD'].groupby('uuid').mean()

In [7]:
# Normal Print Prices
normalprices = pd.merge(normal_retailPrices,normal_buylistPrices,how = 'outer', right_index = True,left_index = True)

normalprices['normal price'] = normalprices['price USD_x'].fillna(normalprices['price USD_y'])
normalprices.drop(['price USD_x','price USD_y'],axis=1,inplace = True)

In [8]:
# Foil Print Prices 
foilprices = pd.merge(foil_retailPrices,foil_buylistPrices,how = 'outer', right_index = True,left_index = True)

foilprices['foil price'] = foilprices['price USD_x'].fillna(foilprices['price USD_y'])
foilprices.drop(['price USD_x','price USD_y'],axis=1,inplace = True)

In [9]:
Prices = pd.merge(normalprices,foilprices,how = 'outer', right_index = True,left_index = True)
Prices

,normal price,foil price
uuid,,
00010d56-fe38-5e35-8aed-518019aa36a5,5.987500,7.460000
0001e0d0-2dcd-5640-aadc-a84765cf5fc9,4.951500,NaN
0003caab-9ff5-5d1a-bc06-976dd0457f19,0.826550,3.405067
0003d249-25d9-5223-af1e-1130f09622a7,0.185300,0.417000
0004a4fb-92c6-59b2-bdbe-ceb584a9e401,1.256850,3.205667
...,...,...
fffbc95a-c4d1-56aa-8653-8a7c71fe19ce,6.034300,7.420000
fffbfc17-a457-565d-a1ed-aefe5810c86a,2.880000,4.359800
fffc1305-a118-559b-9504-3d7b56ca0bde,0.791500,NaN


# All Cards Data

### Clean Data

In [10]:
# Load Card Data 
CardData = pd.read_csv('Data/cards.csv',index_col = 'uuid')
CardData['uuid'] = CardData.index

C:\Users\helal\AppData\Local\Temp\ipykernel_3228\787122630.py:2: DtypeWarning: Columns (3,7,12,16,20,23,47,52,53,61,62,66,68) have mixed types. Specify dtype option on import or set low_memory=False.
  CardData = pd.read_csv('Data/cards.csv',index_col = 'uuid')


In [11]:
#Fill Na in Relevant Columns 
CardData['isOnlineOnly'].fillna(0, inplace = True)
CardData['isOversized'].fillna(0, inplace = True)
CardData['colors'].fillna('C', inplace = True)
CardData['cardParts'].fillna(0, inplace = True)

In [12]:
# Filter out Mtgo & over sized cards Cards 
filt = (CardData['isOnlineOnly'] == 0) & (CardData['isOversized'] == 0)
Cards = CardData[filt].copy()

### Set up Collection Identifier

In [13]:
mask = ~(Cards['cardParts'] == 0)
Cards.loc[mask,'name'] = Cards['name'].str.split('//',expand = True)[0]

In [14]:
Cards['CollectionID'] = Cards['name'].str.strip()+Cards['setCode']
Cards['CollectionID2'] = Cards['CollectionID']+ Cards['number'] 

# Data Merging

In [15]:
# Merge Cards With Prices
Cards = Cards.merge(Prices,how= 'left',right_index = True,left_index = True)

In [16]:
# Merge Cards With Color Order
ColorOrder = pd.read_csv('Data/Order.csv')
Cards = Cards.merge(ColorOrder, how = 'left', on = 'colors')

# Exporting

In [17]:
Cards.set_index('uuid',inplace = True)
Cards.to_csv('CleanData/Cards.csv',index_label= 'uuid')